In [9]:
# Core Python Data Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Plotting
from plotnine import (
    ggplot, aes,
    geom_col, geom_line, geom_smooth,
    facet_wrap,
    scale_y_continuous, scale_x_continuous,
    labs,
    theme, theme_minimal, theme_matplotlib
)

from mizani.breaks import date_breaks
from mizani.formatters import date_format, currency_format

# Misc
from os import mkdir, getcwd, path

from rich import pretty
pretty.install()


ModuleNotFoundError: No module named 'pandas'

In [6]:
import warnings
from openpyxl import Workbook

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
# 2.0 Importing Data Files ----
# help(pd.read_excel)
# - Use "q" to quit

# getcwd()
bikes_df = pd.read_excel(f'./00_data_raw/bikes.xlsx', engine="openpyxl")
bikes_shops_df = pd.read_excel('./00_data_raw/bikeshops.xlsx', engine="openpyxl")
orderlines_df = pd.read_excel(
    io='00_data_raw/orderlines.xlsx',
    converters={'order.date': str},
    engine="openpyxl"
)

# bikes_df.head(10)
# # bike = path.abspath("")
# def get_local_folder():
#     return path.dirname(path.realpath('00_data_raw/bikes.xlsx'))

# rt_fldr = get_local_folder()
# print(f'rel: {rt_fldr}')
bikes_shops_df
bikes_df.head(10)
orderlines_df

In [ ]:
# 3.0 Examining Data ----


# 4.0 Joining Data ----


# 5.0 Wrangling Data ----

# * No copy


# * Copy


# * Handle Dates


# * Show Effect: Copy vs No Copy


# * Text Columns


# * Splitting Description into category_1, category_2, and frame_material


# * Splitting Location into City and State


# * Price Extended


# * Reorganizing


# * Renaming columns


# 6.0 Visualizing a Time Series ----


# 6.1 Total Sales by Month ----


# Quick Plot ----


# Report Plot ----


# 6.2 Sales by Year and Category 2 ----

# ** Step 1 - Manipulate ----


# Step 2 - Visualize ----


# Simple Plot


# Reporting Plot


# 7.0 Writing Files ----


# Pickle ----


# CSV ----


# Excel ----


# WHERE WE'RE GOING
# - Building a forecast system
# - Create a database to host our raw data
# - Develop Modular Functions to:
#   - Collect data
#   - Summarize data and prepare for forecast
#   - Run Automatic Forecasting for One or More Time Series
#   - Store Forecast in Database
#   - Retrieve Forecasts and Report using Templates


In [ ]:
s = bikes_df['description']

In [ ]:
freq_count_series = s.value_counts()

In [ ]:
freq_count_series.nlargest()

In [ ]:
top5_bikes_series = bikes_df['description'].value_counts().nlargest()

In [ ]:
?pd.Series.plot

In [ ]:
top5_bikes_series

In [3]:
fig1 = top5_bikes_series.plot(kind='barh')
fig1.invert_yaxis()

NameError: name 'top5_bikes_series' is not defined

In [ ]:
fig2 = pd.Series.plot(top5_bikes_series)
plt.show()